# Vitessce configs for paper figures

In [ ]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    DataType as dt,
    FileType as ft,
    MultiImageWrapper,
    OmeTiffWrapper,
    BASE_URL_PLACEHOLDER
)
import json
from urllib.parse import quote_plus
from os.path import join
import os

In [ ]:
!pwd

In [ ]:
PATH_TO_EXPORT_DIRECTORY = join("..", "..", "hubmap-publication-page", "data", "ims")
VIGNETTE_DIR = join("..", "..", "hubmap-publication-page", "vignettes", "vignette_03")

In [ ]:
BASE_DIR = join("..", "data")

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name="Spraggins et al.", description="Comparison of aggregation methods for an imaging mass spectrometry dataset")
ds_simple = vc.add_dataset(name="SIMPLE").add_object(
    OmeTiffWrapper(img_path=join(BASE_DIR, "VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.SIMPLE.ome.tiff"), name='SIMPLE')
)
ds_gaussian = vc.add_dataset(name="GAUSSIAN").add_object(
    OmeTiffWrapper(img_path=join(BASE_DIR, "VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.GAUSSIAN.ome.tiff"), name='GAUSSIAN')
)
ds_area = vc.add_dataset(name="AREA").add_object(
    OmeTiffWrapper(img_path=join(BASE_DIR, "VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.AREA.ome.tiff"), name='AREA')
)
ds_linear = vc.add_dataset(name="LINEAR").add_object(
    OmeTiffWrapper(img_path=join(BASE_DIR, "VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.LINEAR.ome.tiff"), name='LINEAR')
)
sp_simple = vc.add_view(vt.SPATIAL, dataset=ds_simple)
sl_simple = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_simple)

sp_gaussian = vc.add_view(vt.SPATIAL, dataset=ds_gaussian)
sl_gaussian = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_gaussian)

sp_area = vc.add_view(vt.SPATIAL, dataset=ds_area)
sl_area = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_area)

sp_linear = vc.add_view(vt.SPATIAL, dataset=ds_linear)
sl_linear = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_linear)

vc.link_views([sp_simple, sp_gaussian, sp_area, sp_linear], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y])

vc.layout(((sp_simple / sl_simple) | (sp_gaussian / sl_gaussian)) | ((sp_area / sl_area) | (sp_linear / sl_linear)));

In [ ]:
#vw = vc.widget(height=800)
#vw

In [ ]:
vc.web_app()

In [ ]:
os.makedirs(VIGNETTE_DIR, exist_ok=True)

In [ ]:
os.makedirs(PATH_TO_EXPORT_DIRECTORY, exist_ok=True)
config_dict = vc.export(to="files", base_url=f"{BASE_URL_PLACEHOLDER}/ims", out_dir=PATH_TO_EXPORT_DIRECTORY)

# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(join(VIGNETTE_DIR, "ims.json"), "w") as f:
    json.dump(config_dict, f)


In [ ]:
vignette_md = """---
name: Use Case 3
figures:
    - name: "Visualization"
      file: ims.json
---

## Comparison of interpolation methods for an imaging mass spectrometry dataset from the human kidney

Vitessce can be used for visual comparison of the outputs of multiple data processing methods within datasets containing multiple technical conditions, supporting the workflows of algorithm developers and computational biologists. For example, computational biologists tasked with analyzing imaging mass spectrometry (IMS) data may be interested in the consequences of using different interpolation methods for generation of pyramidal image files. Pyramidal image files contain downsampled lower resolution versions of the same image which are generated by pre-aggregating pixels, facilitating multi-scale visualization in a web browser. The choice of a downsampling pre-aggregation interpolation function may affect the visual properties of lower resolution images. The open microscopy image processing toolkit bioformats2raw implements several common interpolation methods: SIMPLE, GAUSSIAN, AREA, LINEAR, CUBIC, and LANCZOS. We use the multi-scale image rendering capabilities of Vitessce to compare the application of these methods to an IMS dataset. The zoom levels and centers of each spatial plot are coordinated to facilitate comparison of the same image region at multiple scales. Vitessce helps to discover that the SIMPLE interpolation method introduces regularly-spaced horizontal streaks into the lowest resolution version of the image. Zooming in prompts Vitessce to load a higher resolution image in which the streaks are absent, indicating that they are artifacts of the downsampling procedure.  This insight may steer us away from using the SIMPLE method when processing IMS data.

"""

with open(join(VIGNETTE_DIR, "description.md"), "w") as f:
    f.write(vignette_md)